In [1]:
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Optional
from pathlib import Path
from datetime import datetime, timedelta
from enum import Enum
import logging
from scipy import stats
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
import warnings
import sys
import time
import json
import traceback
warnings.filterwarnings('ignore')

In [2]:
class IndexType(Enum):
    """指數類型"""
    PRICE = "價格指數"    # 一般類指數
    RETURN = "報酬指數"   # 類報酬指數
    LEVERAGE = "槓桿指數"  # 兩倍槓桿指數
    INVERSE = "反向指數"   # 反向指數

class TimeFrequency(Enum):
    """時間頻率"""
    WEEKLY = 'W'
    MONTHLY = 'ME'

In [3]:
class IndustryAnalysisSystem:
    """產業分析系統"""
    
    def __init__(self, base_path: str = "D:/Min/Python/Project/FA_Data"):
        self.base_path = Path(base_path)
        self.logger = logging.getLogger(self.__class__.__name__)
        
        # 建立必要的目錄結構
        self._initialize_directories()
        
        try:
            # 載入基礎資料
            self.company_data = pd.read_csv(self.base_path / "meta_data/companies_final.csv")
            self.logger.info("成功載入公司資料")
            
            self.industry_index = pd.read_csv(self.base_path / "meta_data/industry_index.csv")
            self.logger.info("成功載入產業指數資料")
            
            self.market_index = pd.read_csv(self.base_path / "meta_data/market_index.csv")
            self.logger.info("成功載入市場指數資料")
            
            # 處理日期格式
            self.industry_index['日期'] = pd.to_datetime(self.industry_index['日期'])
            self.market_index['Date'] = pd.to_datetime(self.market_index['Date'])
            
            # 建立產業對應關係
            self.industry_mapping = self._create_industry_mapping()
            
            # 驗證產業指數配對
            validation_results = self.validate_index_pairs()
            if validation_results['missing_pairs'] or validation_results['incomplete_pairs']:
                self.logger.warning("發現不完整的產業指數配對:")
                if validation_results['missing_pairs']:
                    self.logger.warning(f"缺少配對的產業: {validation_results['missing_pairs']}")
                if validation_results['incomplete_pairs']:
                    self.logger.warning(f"不完整配對的產業: {validation_results['incomplete_pairs']}")
                if validation_results['special_indices']:
                    self.logger.info(f"特殊指數產業: {validation_results['special_indices']}")
                    
        except FileNotFoundError as e:
            self.logger.error(f"找不到必要的資料檔案: {str(e)}")
            raise
        except pd.errors.EmptyDataError as e:
            self.logger.error(f"資料檔案是空的: {str(e)}")
            raise
        except Exception as e:
            self.logger.error(f"初始化時發生未預期的錯誤: {str(e)}")
            raise

    def _initialize_directories(self):
        """初始化目錄結構"""
        directories = [
            "industry_correlation/weekly",
            "industry_correlation/monthly",
            "industry_analysis/price_index",
            "industry_analysis/return_index",
            "meta_data/backup"
        ]
        
        for directory in directories:
            (self.base_path / directory).mkdir(parents=True, exist_ok=True)

    def get_industry_stocks(self, industry_name: str) -> List[str]:
        """獲取特定產業的所有股票"""
        base_name = industry_name.replace('類報酬指數', '').replace('類指數', '').strip()
        all_stocks = set()
        
        if base_name in self.industry_mapping:
            # 獲取所有相關的產業類別
            categories = self.industry_mapping[base_name]['categories']
            
            # 對每個類別獲取股票
            for category in categories:
                stocks = self.company_data[
                    self.company_data['industry_category'] == category
                ]['stock_id'].unique()
                all_stocks.update(stocks)
        
        return list(all_stocks)

    def validate_index_pairs(self) -> Dict:
        """驗證產業指數配對完整性"""
        validation_results = {
            'missing_pairs': [],      # 缺少配對的產業
            'incomplete_pairs': [],   # 不完整配對的產業
            'special_indices': []     # 有特殊指數的產業
        }
        
        for base_name, indices in self.industry_mapping.items():
            # 檢查基本配對
            has_price = bool(indices[IndexType.PRICE])
            has_return = bool(indices[IndexType.RETURN])
            has_leverage = bool(indices[IndexType.LEVERAGE])
            has_inverse = bool(indices[IndexType.INVERSE])
            
            # 完全缺少指數的產業
            if not any([has_price, has_return, has_leverage, has_inverse]):
                validation_results['missing_pairs'].append(base_name)
                continue
                
            # 檢查價格和報酬指數配對
            if has_price != has_return:
                validation_results['incomplete_pairs'].append({
                    'industry': base_name,
                    'has_price': has_price,
                    'has_return': has_return
                })
            
            # 檢查特殊指數
            if has_leverage or has_inverse:
                validation_results['special_indices'].append({
                    'industry': base_name,
                    'has_leverage': has_leverage,
                    'has_inverse': has_inverse
                })
        
        return validation_results
    
    def get_industry_performance(self, 
                               industry_name: str, 
                               index_type: IndexType = None,
                               start_date: str = None, 
                               end_date: str = None) -> pd.DataFrame:
        """獲取產業指數表現"""
        base_name = industry_name.replace('類報酬指數', '').replace('類指數', '').strip()
        
        if base_name in self.industry_mapping:
            # 根據指數類型選擇指數
            if index_type is None:
                # 如果未指定類型，優先使用報酬指數
                index_names = (self.industry_mapping[base_name][IndexType.RETURN] or 
                             self.industry_mapping[base_name][IndexType.PRICE])
            else:
                index_names = self.industry_mapping[base_name][index_type]
                
            if not index_names:
                return pd.DataFrame()
            
            # 獲取所有相關指數的數據
            industry_data = self.industry_index[
                self.industry_index['指數名稱'].isin(index_names)
            ].copy()
            
            # 日期過濾
            if start_date:
                industry_data = industry_data[
                    industry_data['日期'] >= pd.to_datetime(start_date)
                ]
            if end_date:
                industry_data = industry_data[
                    industry_data['日期'] <= pd.to_datetime(end_date)
                ]
            
            if not industry_data.empty:
                # 計算相關指標
                industry_data['daily_return'] = industry_data.groupby('指數名稱')['收盤指數'].pct_change()
                industry_data['volatility'] = industry_data.groupby('指數名稱')['daily_return'].rolling(20).std().values
            
            return industry_data
        
        return pd.DataFrame()
    
    def _create_industry_mapping(self) -> Dict:
        """建立產業分類對應關係"""
        mapping = {}
        
        # 建立指數類型分類
        def get_index_type(name: str) -> IndexType:
            """判斷指數類型"""
            if '報酬指數' in name:
                return IndexType.RETURN
            elif '兩倍槓桿指數' in name or '日報酬兩倍指數' in name:
                return IndexType.LEVERAGE
            elif '反向指數' in name or '反向一倍指數' in name:
                return IndexType.INVERSE
            else:
                return IndexType.PRICE
        
        index_types = {
            name: get_index_type(name)
            for name in self.industry_index['指數名稱'].unique()
        }
        
        # 產業名稱標準化
        def get_base_name(name: str) -> str:
            """獲取基礎產業名稱"""
            return name.replace('類報酬指數', '')\
                      .replace('類指數', '')\
                      .replace('類日報酬兩倍指數', '')\
                      .replace('類日報酬反向一倍指數', '')\
                      .replace('類兩倍槓桿指數', '')\
                      .replace('類反向指數', '')\
                      .strip()
        
        # 建立產業分類到指數的映射
        for index_name in self.industry_index['指數名稱'].unique():
            base_name = get_base_name(index_name)
            if base_name not in mapping:
                mapping[base_name] = {
                    IndexType.PRICE: [],
                    IndexType.RETURN: [],
                    IndexType.LEVERAGE: [],
                    IndexType.INVERSE: [],
                    'categories': set()
                }
            
            # 根據指數類型分類
            index_type = index_types[index_name]
            mapping[base_name][index_type].append(index_name)
        
        # 添加產業類別對應關係
        for category in self.company_data['industry_category'].unique():
            base_name = get_base_name(category)
            if base_name in mapping:
                mapping[base_name]['categories'].add(category)
            
            # 特殊處理：如果類別名稱本身就是一個完整的產業名稱
            if category.endswith('業'):
                base_name_from_category = get_base_name(category)
                if base_name_from_category in mapping:
                    mapping[base_name_from_category]['categories'].add(category)
        
        # 將set轉換回list
        for base_name in mapping:
            mapping[base_name]['categories'] = list(mapping[base_name]['categories'])
        
        return mapping

    def analyze_time_series_patterns(self, 
                                  industry_name: str,
                                  start_date: Optional[str] = None,
                                  end_date: Optional[str] = None,
                                  index_type: Optional[IndexType] = None) -> Dict:
        """分析產業指數的時間序列特性"""
        result = {
            'trend': {},
            'seasonality': {},
            'lead_lag': {}
        }
        
        # 根據指定的指數類型獲取數據
        if index_type:
            industry_data = self.get_industry_performance(
                industry_name,
                index_type=index_type,
                start_date=start_date,
                end_date=end_date
            )
        else:
            # 優先使用報酬指數
            industry_data = self.get_industry_performance(
                industry_name,
                index_type=IndexType.RETURN,
                start_date=start_date,
                end_date=end_date
            )
            
            # 如果沒有報酬指數,則使用價格指數
            if industry_data.empty:
                industry_data = self.get_industry_performance(
                    industry_name,
                    index_type=IndexType.PRICE,
                    start_date=start_date,
                    end_date=end_date
                )
        
        if industry_data.empty:
            return result
        
        # 記錄使用的指數類型
        index_type = IndexType.RETURN if '報酬指數' in industry_data['指數名稱'].iloc[0] else IndexType.PRICE
        result['index_type'] = index_type.value
        
        # 計算趨勢分析
        result['trend'] = self._analyze_trend(industry_data)
        
        # 計算季節性分析
        result['seasonality'] = self._analyze_seasonality(industry_data)
        
        # 計算領先/落後關係
        result['lead_lag'] = self._analyze_lead_lag_relationship(industry_data)
        
        return result

    def _analyze_trend(self, data: pd.DataFrame) -> Dict:
        """分析趨勢特性"""
        trend_analysis = {}
        
        try:
            # 確保數據已經按日期排序
            data = data.sort_values('日期')
            
            # 計算線性趨勢
            X = np.arange(len(data)).reshape(-1, 1)
            y = data['收盤指數'].values
            model = LinearRegression()
            model.fit(X, y)
            
            # 計算趨勢斜率
            trend_analysis['slope'] = float(model.coef_[0])
            
            # 判斷趨勢方向
            trend_analysis['trend_direction'] = 'upward' if model.coef_[0] > 0 else 'downward'
            
            # 計算R平方值
            trend_analysis['r2_score'] = float(model.score(X, y))
            
            # 計算趨勢強度（修正後的計算方法）
            # 使用斜率除以平均價格，再乘以資料天數來表示整體趨勢強度
            mean_price = np.mean(y)
            days = len(data)
            if mean_price != 0:
                trend_strength = abs(model.coef_[0] * days / mean_price)
                trend_analysis['trend_strength'] = float(trend_strength)
            else:
                trend_analysis['trend_strength'] = 0.0
                
            # 加入其他趨勢指標
            trend_analysis['price_range'] = {
                'start_price': float(y[0]),
                'end_price': float(y[-1]),
                'total_return': float((y[-1] / y[0] - 1) * 100)  # 總報酬率%
            }
            
        except Exception as e:
            self.logger.error(f"計算趨勢時發生錯誤: {str(e)}")
            trend_analysis = {
                'slope': 0.0,
                'trend_direction': 'unknown',
                'r2_score': 0.0,
                'trend_strength': 0.0,
                'price_range': {'start_price': 0, 'end_price': 0, 'total_return': 0}
            }
        
        return trend_analysis
    
    def _analyze_seasonality(self, data: pd.DataFrame) -> Dict:
        """分析季節性特性"""
        seasonality = {}
        
        # 添加月份和季度信息
        data = data.copy()
        data['month'] = data['日期'].dt.month
        data['quarter'] = data['日期'].dt.quarter
        
        # 月度季節性
        monthly_means = data.groupby('month')['收盤指數'].mean()
        monthly_std = data.groupby('month')['收盤指數'].std()
        
        seasonality['monthly'] = {
            'strongest_month': monthly_means.idxmax(),
            'weakest_month': monthly_means.idxmin(),
            'monthly_pattern': monthly_means.to_dict(),
            'monthly_volatility': monthly_std.to_dict()
        }
        
        # 季度季節性
        quarterly_means = data.groupby('quarter')['收盤指數'].mean()
        quarterly_std = data.groupby('quarter')['收盤指數'].std()
        
        seasonality['quarterly'] = {
            'strongest_quarter': quarterly_means.idxmax(),
            'weakest_quarter': quarterly_means.idxmin(),
            'quarterly_pattern': quarterly_means.to_dict(),
            'quarterly_volatility': quarterly_std.to_dict()
        }
        
        return seasonality
    
    def _analyze_lead_lag_relationship(self, data: pd.DataFrame) -> Dict:
        """分析領先/落後關係"""
        lead_lag = {}
        
        # 與大盤指數的領先/落後關係
        market_data = self.market_index.copy()
        market_data = market_data.rename(columns={'Date': '日期'})
        
        # 合併數據
        merged_data = pd.merge(
            data[['日期', '收盤指數']],
            market_data[['日期', 'Close']],
            on='日期'
        )
        
        # 計算收益率
        merged_data['industry_return'] = merged_data['收盤指數'].pct_change()
        merged_data['market_return'] = merged_data['Close'].pct_change()
        
        # 計算不同lag的相關性
        correlations = {}
        for lag in range(-10, 11):  # 檢查前後10天
            if lag < 0:
                corr = merged_data['industry_return'].shift(abs(lag)).corr(merged_data['market_return'])
                correlations[lag] = corr
            else:
                corr = merged_data['industry_return'].corr(merged_data['market_return'].shift(lag))
                correlations[lag] = corr
        
        # 找出最強的領先/落後關係
        max_corr_lag = max(correlations.items(), key=lambda x: abs(x[1]))
        
        lead_lag['max_correlation_lag'] = max_corr_lag[0]
        lead_lag['max_correlation_value'] = max_corr_lag[1]
        lead_lag['relationship_type'] = 'leading' if max_corr_lag[0] < 0 else 'lagging' if max_corr_lag[0] > 0 else 'concurrent'
        lead_lag['all_correlations'] = correlations
        
        return lead_lag

    def analyze_risk_metrics(self, 
                           industry_name: str,
                           start_date: Optional[str] = None,
                           end_date: Optional[str] = None,
                           index_type: Optional[IndexType] = None) -> Dict:
        """分析風險指標"""
        try:
            # 根據指定的指數類型獲取數據
            if index_type:
                industry_data = self.get_industry_performance(
                    industry_name,
                    index_type=index_type,
                    start_date=start_date,
                    end_date=end_date
                )
            else:
                # 優先使用報酬指數
                industry_data = self.get_industry_performance(
                    industry_name,
                    index_type=IndexType.RETURN,
                    start_date=start_date,
                    end_date=end_date
                )
                
                # 如果沒有報酬指數，則使用價格指數
                if industry_data.empty:
                    industry_data = self.get_industry_performance(
                        industry_name,
                        index_type=IndexType.PRICE,
                        start_date=start_date,
                        end_date=end_date
                    )
            
            if industry_data.empty:
                return {}
            
            # 計算日收益率
            returns = industry_data['收盤指數'].pct_change()
            
            # 計算各項風險指標
            risk_metrics = self.calculate_risk_metrics(returns)
            
            # 添加回撤分析
            risk_metrics['drawdown'] = self._calculate_drawdown_metrics(industry_data['收盤指數'])
            
            # 記錄使用的指數類型
            risk_metrics['index_type'] = IndexType.RETURN.value if '報酬指數' in industry_data['指數名稱'].iloc[0] else IndexType.PRICE.value
            
            return risk_metrics
            
        except Exception as e:
            self.logger.error(f"計算風險指標時發生錯誤: {str(e)}")
            return {}
    
    def _calculate_downside_risk(self, returns: pd.Series) -> Dict:
        """計算下跌風險指標"""
        downside = {}
        
        # 下跌風險
        negative_returns = returns[returns < 0]
        downside['avg_loss'] = negative_returns.mean()
        downside['loss_std'] = negative_returns.std()
        downside['loss_frequency'] = len(negative_returns) / len(returns)
        
        # 下跌波動率
        downside_returns = returns[returns < 0]
        downside['downside_volatility'] = np.sqrt(np.sum(downside_returns ** 2) / len(returns)) * np.sqrt(252)
        
        return downside
    
    def _calculate_drawdown_metrics(self, prices: pd.Series) -> Dict:
        """計算回撤相關指標"""
        drawdown = {}
        
        # 計算回撤序列
        roll_max = prices.expanding().max()
        drawdowns = (prices - roll_max) / roll_max
        
        # 最大回撤
        drawdown['max_drawdown'] = drawdowns.min()
        
        # 平均回撤
        drawdown['avg_drawdown'] = drawdowns[drawdowns < 0].mean()
        
        # 回撤持續時間分析
        drawdown_periods = self._analyze_drawdown_periods(prices)
        drawdown['avg_recovery_time'] = drawdown_periods['avg_recovery_time']
        drawdown['max_recovery_time'] = drawdown_periods['max_recovery_time']
        
        return drawdown
    
    def _analyze_drawdown_periods(self, prices: pd.Series) -> Dict:
        """分析回撤期間"""
        result = {'avg_recovery_time': 0, 'max_recovery_time': 0}
        
        # 確保索引連續
        prices = prices.reset_index(drop=True)
        
        high_water_mark = prices.expanding().max()
        drawdowns = (prices - high_water_mark) / high_water_mark
        
        # 找出回撤開始和結束點
        in_drawdown = False
        drawdown_starts = []
        recovery_times = []
        
        # 使用 itertuples() 或 enumerate 來避免索引問題
        for i, drawdown in enumerate(drawdowns):
            if not in_drawdown and drawdown < 0:
                in_drawdown = True
                drawdown_starts.append(i)
            elif in_drawdown and prices.iloc[i] >= high_water_mark.iloc[i]:
                in_drawdown = False
                recovery_times.append(i - drawdown_starts[-1])
        
        if recovery_times:
            result['avg_recovery_time'] = np.mean(recovery_times)
            result['max_recovery_time'] = max(recovery_times)
        
        return result

    def calculate_risk_metrics(self, returns: pd.Series, risk_free_rate: float = 0.02) -> Dict:
        """整合計算風險指標"""
        metrics = {}
        
        # 基礎風險指標計算
        annual_return = returns.mean() * 252
        annual_volatility = returns.std() * np.sqrt(252)
        
        # 風險調整後收益比率
        metrics['ratios'] = {
            'annual_return': annual_return,
            'annual_volatility': annual_volatility,
            'sharpe_ratio': (annual_return - risk_free_rate) / annual_volatility if annual_volatility != 0 else 0,
        }
        
        # 下檔風險指標
        downside_returns = returns[returns < 0]
        downside_volatility = downside_returns.std() * np.sqrt(252)
        
        metrics['downside'] = {
            'downside_volatility': downside_volatility,
            'sortino_ratio': (annual_return - risk_free_rate) / downside_volatility if downside_volatility != 0 else 0,
            'loss_frequency': len(downside_returns) / len(returns),
            'avg_loss': downside_returns.mean()
        }
        
        # 尾部風險指標
        metrics['tail_risk'] = {
            'var_95': returns.quantile(0.05),
            'var_99': returns.quantile(0.01),
            'cvar_95': returns[returns <= returns.quantile(0.05)].mean(),
            'cvar_99': returns[returns <= returns.quantile(0.01)].mean(),
            'skewness': returns.skew(),
            'kurtosis': returns.kurtosis()
        }
        
        return metrics

    def analyze_industry_rotation(self, 
                                lookback_period: int = 120,
                                momentum_period: int = 20) -> Dict:
        """分析產業輪動"""
        rotation_analysis = {
            'strength_ranking': {},
            'momentum_ranking': {},
            'flow_analysis': {},
            'rotation_cycles': {}
        }
        
        # 獲取所有產業的數據
        industry_data = self.industry_index.copy()
        industry_data['base_industry'] = industry_data['指數名稱'].apply(
            lambda x: x.replace('類報酬指數', '').replace('類指數', '').strip()
        )
        
        # 計算產業強弱勢排名
        rotation_analysis['strength_ranking'] = self._analyze_industry_strength(
            industry_data,
            lookback_period
        )
        
        # 計算產業動能排名
        rotation_analysis['momentum_ranking'] = self._analyze_industry_momentum(
            industry_data,
            momentum_period
        )
        
        # 分析資金流向
        rotation_analysis['flow_analysis'] = self._analyze_fund_flow(
            industry_data,
            lookback_period
        )
        
        # 分析輪動週期
        rotation_analysis['rotation_cycles'] = self._analyze_rotation_cycles(
            industry_data
        )
        
        return rotation_analysis

    def _analyze_industry_strength(self, data: pd.DataFrame, lookback_period: int) -> Dict:
        """分析產業強弱勢"""
        strength = {}
        
        # 計算各產業最近期的表現
        latest_date = data['日期'].max()
        start_date = latest_date - pd.Timedelta(days=lookback_period)
        
        period_data = data[data['日期'] >= start_date].copy()
        
        # 計算期間報酬率
        period_returns = {}
        for industry in period_data['base_industry'].unique():
            industry_data = period_data[period_data['base_industry'] == industry]
            if not industry_data.empty:
                start_price = industry_data.loc[industry_data['日期'].idxmin(), '收盤指數']
                end_price = industry_data.loc[industry_data['日期'].idxmax(), '收盤指數']
                period_returns[industry] = (end_price / start_price - 1) * 100
        
        # 計算相對強度指標
        strength['period_returns'] = period_returns
        strength['ranking'] = pd.Series(period_returns).sort_values(ascending=False)
        
        # 計算強弱分界
        returns_series = pd.Series(period_returns)
        strength['strong_industries'] = returns_series[returns_series > returns_series.mean()].index.tolist()
        strength['weak_industries'] = returns_series[returns_series <= returns_series.mean()].index.tolist()
        
        return strength
    
    def _analyze_industry_momentum(self, data: pd.DataFrame, momentum_period: int) -> Dict:
        """分析產業動能"""
        momentum = {}
        
        latest_date = data['日期'].max()
        start_date = latest_date - pd.Timedelta(days=momentum_period)
        
        # 計算動能指標
        momentum_scores = {}
        for industry in data['base_industry'].unique():
            industry_data = data[
                (data['base_industry'] == industry) & 
                (data['日期'] >= start_date)
            ].copy()
            
            if not industry_data.empty:
                # 計算價格動能
                price_momentum = industry_data['收盤指數'].pct_change().mean()
                
                # 計算成交量動能 (如果有成交量數據)
                volume_momentum = 0
                if '成交金額' in industry_data.columns:
                    volume_momentum = industry_data['成交金額'].pct_change().mean()
                
                # 綜合動能分數
                momentum_scores[industry] = {
                    'price_momentum': price_momentum,
                    'volume_momentum': volume_momentum,
                    'composite_score': price_momentum + 0.5 * volume_momentum
                }
        
        # 動能排名
        composite_scores = {k: v['composite_score'] for k, v in momentum_scores.items()}
        momentum['scores'] = momentum_scores
        momentum['ranking'] = pd.Series(composite_scores).sort_values(ascending=False)
        
        return momentum
    
    def _analyze_fund_flow(self, data: pd.DataFrame, lookback_period: int) -> Dict:
        """資金流向分析"""
        # 準備數據
        latest_date = data['日期'].max()
        start_date = latest_date - pd.Timedelta(days=lookback_period)
        period_data = data[data['日期'] >= start_date].copy()
        
        # 計算基本指標
        period_data['daily_return'] = period_data.groupby('base_industry')['收盤指數'].pct_change()
        period_data['volume_change'] = period_data.groupby('base_industry')['成交金額'].pct_change() if '成交金額' in period_data.columns else 0
        
        flow_analysis = {}
        
        # 按產業分組計算
        for industry in period_data['base_industry'].unique():
            industry_data = period_data[period_data['base_industry'] == industry]
            
            # 計算綜合指標
            price_momentum = industry_data['daily_return'].mean()
            volume_momentum = industry_data['volume_change'].mean() if isinstance(industry_data['volume_change'], pd.Series) else 0
            volatility = industry_data['daily_return'].std()
            
            flow_score = price_momentum * (1 + 0.5 * abs(volume_momentum))
            
            flow_analysis[industry] = {
                'momentum': price_momentum,
                'volume_trend': volume_momentum,
                'volatility': volatility,
                'flow_score': flow_score
            }
        
        # 轉換為 DataFrame 方便排序和分析
        flow_df = pd.DataFrame.from_dict(flow_analysis, orient='index')
        
        return {
            'indicators': flow_analysis,
            'inflow_industries': flow_df[flow_df['flow_score'] > 0].sort_values('flow_score', ascending=False).index.tolist(),
            'outflow_industries': flow_df[flow_df['flow_score'] < 0].sort_values('flow_score').index.tolist(),
            'rankings': {
                'momentum': flow_df.sort_values('momentum', ascending=False).index.tolist(),
                'volume': flow_df.sort_values('volume_trend', ascending=False).index.tolist()
            }
        }
    
    def _analyze_rotation_cycles(self, data: pd.DataFrame) -> Dict:
        """分析輪動週期"""
        cycles = {}
        
        # 計算每個產業的相對強度
        industry_returns = {}
        for industry in data['base_industry'].unique():
            industry_data = data[data['base_industry'] == industry].copy()
            if not industry_data.empty:
                industry_data['returns'] = industry_data['收盤指數'].pct_change()
                industry_returns[industry] = industry_data['returns']
        
        # 計算產業輪動特徵
        rotation_features = {}
        for industry, returns in industry_returns.items():
            if len(returns) > 0:
                # 計算產業週期特徵
                rotation_features[industry] = {
                    'mean_return': returns.mean(),
                    'volatility': returns.std(),
                    'skewness': returns.skew(),
                    'autocorrelation': returns.autocorr()
                }
        
        cycles['features'] = rotation_features
        
        # 識別當前輪動階段
        latest_returns = {industry: returns.iloc[-20:].mean() for industry, returns in industry_returns.items()}
        sorted_industries = pd.Series(latest_returns).sort_values(ascending=False)
        
        # 定義輪動階段
        top_quartile = sorted_industries[:len(sorted_industries)//4]
        cycles['current_leaders'] = top_quartile.index.tolist()
        
        # 計算產業間的領先/落後關係
        lead_lag_matrix = {}
        for ind1 in industry_returns.keys():
            lead_lag_matrix[ind1] = {}
            for ind2 in industry_returns.keys():
                if ind1 != ind2:
                    correlation = industry_returns[ind1].corr(industry_returns[ind2])
                    lead_lag_matrix[ind1][ind2] = correlation
        
        cycles['lead_lag_matrix'] = lead_lag_matrix
        
        return cycles

    def generate_enhanced_report(self, 
                               industry_name: str,
                               start_date: Optional[str] = None,
                               end_date: Optional[str] = None,
                               index_type: Optional[IndexType] = None) -> Dict:  # 加入 index_type 參數
        """生成增強版產業分析報告"""
        report = {
            'basic_info': {},
            'time_series_analysis': {},
            'risk_analysis': {},
            'rotation_analysis': {},
            'technical_analysis': {},
            'investment_suggestions': {}
        }
        
        # 基本信息
        report['basic_info'] = {
            'industry_name': industry_name,
            'period': {'start': start_date, 'end': end_date},
            'stocks': self.get_industry_stocks(industry_name)
        }
        
        # 時間序列分析
        report['time_series_analysis'] = self.analyze_time_series_patterns(
            industry_name, 
            start_date, 
            end_date,
            index_type=index_type  # 傳遞 index_type 參數
        )
        
        # 風險分析
        report['risk_analysis'] = self.analyze_risk_metrics(
            industry_name, 
            start_date, 
            end_date,
            index_type=index_type  # 傳遞 index_type 參數
        )
        
        # 產業輪動分析
        report['rotation_analysis'] = self.analyze_industry_rotation()
        
        # 生成投資建議
        report['investment_suggestions'] = self._generate_investment_suggestions(report)
        
        # 記錄使用的指數類型
        report['metadata'] = {
            'index_type': index_type.value if index_type else 'auto_selected',
            'generation_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
        
        return report
    
    def _generate_investment_suggestions(self, analysis_report: Dict) -> Dict:
        """生成投資建議"""
        # 從 basic_info 中獲取產業名稱
        industry_name = analysis_report.get('basic_info', {}).get('industry_name', '')
        suggestions = {
            'risk_assessment': '',
            'timing_suggestions': '',
            'position_suggestions': '',
            'key_points': []
        }
        
        try:
            # 安全地獲取風險分析數據
            risk_analysis = analysis_report.get('risk_analysis', {})
            ratios = risk_analysis.get('ratios', {})
            drawdown = risk_analysis.get('drawdown', {})
            
            # 評估風險水平
            sharpe_ratio = ratios.get('sharpe_ratio', 0)
            if sharpe_ratio > 1:
                risk_level = '較低'
            elif sharpe_ratio > 0:
                risk_level = '中等'
            else:
                risk_level = '較高'
                
            suggestions['risk_assessment'] = f"當前風險水平: {risk_level}"
            
            # 時間序列分析建議
            time_series = analysis_report.get('time_series_analysis', {})
            trend = time_series.get('trend', {})
            trend_direction = trend.get('trend_direction', '')
            trend_strength = trend.get('trend_strength', 0)
            
            if trend_direction == 'upward':
                if trend_strength > 0.1:
                    suggestions['timing_suggestions'] = "當前處於強勢上升趨勢，可考慮逢回調布局"
                else:
                    suggestions['timing_suggestions'] = "當前處於溫和上升趨勢，建議分批布局"
            else:
                suggestions['timing_suggestions'] = "當前處於下降趨勢，建議觀望或降低持倉"
            
            # 產業輪動建議
            rotation = analysis_report.get('rotation_analysis', {})
            strength_ranking = rotation.get('strength_ranking', {})
            strong_industries = strength_ranking.get('strong_industries', [])
            
            if industry_name in strong_industries:
                suggestions['position_suggestions'] = "產業處於強勢地位，可維持較高持倉"
            else:
                suggestions['position_suggestions'] = "產業強度偏弱，建議控制持倉比例"
            
            # 關鍵要點
            max_drawdown = drawdown.get('max_drawdown', 0)
            ranking = -1
            if 'ranking' in strength_ranking:
                try:
                    ranking = strength_ranking['ranking'].index.get_loc(industry_name) + 1
                except (KeyError, ValueError, AttributeError):
                    ranking = -1
            
            suggestions['key_points'] = [
                f"夏普比率: {sharpe_ratio:.2f}",
                f"最大回撤: {max_drawdown:.2%}",
                f"產業相對強度排名: {ranking if ranking > 0 else '無法確定'}",
                f"建議持倉水位: {'70-90%' if risk_level == '較低' else '50-70%' if risk_level == '中等' else '30-50%'}"
            ]
        
        except Exception as e:
            self.logger.error(f"生成投資建議時發生錯誤: {str(e)}")
            # 設置預設建議
            suggestions['risk_assessment'] = "無法評估風險水平"
            suggestions['timing_suggestions'] = "建議進一步分析後再做決定"
            suggestions['position_suggestions'] = "建議謹慎操作"
            suggestions['key_points'] = ["暫無詳細分析數據"]
        
        return suggestions

    def generate_correlation_matrix(self, date: str) -> pd.DataFrame:
       """生成特定日期的產業相關性矩陣"""
       try:
           # 確保日期格式正確
           date = pd.to_datetime(date)
           
           # 獲取計算相關性的時間範圍(使用前20個交易日的數據)
           start_date = date - pd.Timedelta(days=30)
           
           # 獲取時間範圍內的所有產業數據
           historical_data = self.industry_index[
               (self.industry_index['日期'] >= start_date) & 
               (self.industry_index['日期'] <= date)
           ].copy()
           
           if historical_data.empty:
               print(f"日期 {date.strftime('%Y-%m-%d')} 無可用數據")
               return pd.DataFrame()
           
           # 優先使用報酬指數,如果沒有則使用價格指數
           def get_preferred_index(group):
               return_index = group[group['指數名稱'].str.contains('報酬指數')]
               if not return_index.empty:
                   return return_index
               return group[~group['指數名稱'].str.contains('報酬指數')]
           
           # 標準化產業名稱並選擇合適的指數類型
           historical_data['base_industry'] = historical_data['指數名稱'].apply(
               lambda x: x.replace('類報酬指數', '').replace('類指數', '').strip()
           )
           
           # 對每個產業選擇合適的指數類型
           selected_data = []
           for industry in historical_data['base_industry'].unique():
               industry_group = historical_data[historical_data['base_industry'] == industry]
               preferred_index = get_preferred_index(industry_group)
               selected_data.append(preferred_index)
           
           historical_data = pd.concat(selected_data)
           
           # 計算每個產業的日收益率
           historical_data['daily_return'] = historical_data.groupby('base_industry')['收盤指數'].pct_change()
           
           # 構建收益率數據框
           industries = historical_data['base_industry'].unique()
           returns_data = pd.DataFrame(index=historical_data['日期'].unique())
           
           for industry in industries:
               industry_returns = historical_data[historical_data['base_industry'] == industry]
               returns_data[industry] = industry_returns.set_index('日期')['daily_return']
           
           # 計算相關性矩陣
           corr_matrix = returns_data.corr(method='pearson')
           
           # 確保對角線為1,且矩陣對稱
           np.fill_diagonal(corr_matrix.values, 1)
           corr_matrix = (corr_matrix + corr_matrix.T) / 2
           
           # 儲存相關性矩陣
           date_str = date.strftime('%Y%m%d')
           
           # 確定儲存目錄
           save_dir = self.base_path / "industry_correlation" / "weekly"
           if pd.Timestamp(date).is_month_end:
               save_dir = self.base_path / "industry_correlation" / "monthly"
               
           save_dir.mkdir(parents=True, exist_ok=True)
           
           # 儲存檔案
           save_path = save_dir / f"industry_correlation_{date_str}.csv"
           corr_matrix.to_csv(save_path, encoding='utf-8-sig')
           
           # 確認檔案是否成功儲存
           if not save_path.exists():
               print(f"警告: 檔案未成功儲存到 {save_path}")
           
           return corr_matrix
           
       except Exception as e:
           print(f"生成相關性矩陣時發生錯誤: {str(e)}")
           return pd.DataFrame()

    def analyze_all_industries(self,
                             start_date: Optional[str] = None,
                             end_date: Optional[str] = None) -> Dict[str, Dict]:
        """分析所有產業的報告"""
        reports = {}
        try:
            # 獲取所有基礎產業名稱（去除'類報酬指數'和'類指數'後綴）
            industry_bases = set(self.industry_index['指數名稱'].apply(
                lambda x: x.replace('類報酬指數', '').replace('類指數', '').strip()
            ))
            
            for industry_name in industry_bases:
                reports[industry_name] = {
                    'price': None,
                    'return': None
                }
                
                # 生成價格指數報告
                price_report = self.generate_enhanced_report(
                    industry_name,
                    start_date=start_date,
                    end_date=end_date,
                    index_type=IndexType.PRICE
                )
                if price_report:
                    self.save_industry_report(price_report, industry_name, IndexType.PRICE)
                    reports[industry_name]['price'] = price_report
                    
                # 生成報酬指數報告
                return_report = self.generate_enhanced_report(
                    industry_name,
                    start_date=start_date,
                    end_date=end_date,
                    index_type=IndexType.RETURN
                )
                if return_report:
                    self.save_industry_report(return_report, industry_name, IndexType.RETURN)
                    reports[industry_name]['return'] = return_report
                
                self.logger.info(f"完成產業 {industry_name} 的分析")
                
        except Exception as e:
            self.logger.error(f"分析全產業時發生錯誤: {str(e)}")
            raise
            
        return reports

    def save_industry_report(self, report: Dict, industry_name: str, index_type: IndexType):
        """儲存產業分析報告"""
        try:
            # 決定儲存路徑
            save_dir = self.base_path / "industry_analysis"
            if index_type == IndexType.PRICE:
                save_dir = save_dir / "price_index"
            else:
                save_dir = save_dir / "return_index"
                
            # 確保目錄存在
            save_dir.mkdir(parents=True, exist_ok=True)
            
            # 產生檔名
            current_date = datetime.now().strftime('%Y%m%d')
            period_start = report['basic_info']['period']['start'].replace('-', '') if report['basic_info']['period']['start'] else 'NA'
            period_end = report['basic_info']['period']['end'].replace('-', '') if report['basic_info']['period']['end'] else 'NA'
            
            filename = f"{industry_name}_{period_start}_{period_end}_{current_date}.json"
            save_path = save_dir / filename
            
            def convert_to_serializable(obj):
                """將物件轉換為可序列化的格式"""
                if isinstance(obj, (np.int32, np.int64)):
                    return int(obj)
                elif isinstance(obj, (np.float32, np.float64)):
                    return float(obj)
                elif isinstance(obj, np.ndarray):
                    return obj.tolist()
                elif isinstance(obj, (datetime, pd.Timestamp)):
                    return obj.strftime('%Y-%m-%d')
                elif isinstance(obj, dict):
                    return {k: convert_to_serializable(v) for k, v in obj.items()}
                elif isinstance(obj, (list, tuple)):
                    return [convert_to_serializable(i) for i in obj]
                elif isinstance(obj, (pd.Series, pd.DataFrame)):
                    return convert_to_serializable(obj.to_dict())
                elif pd.isna(obj):
                    return None
                elif isinstance(obj, IndexType):  # 處理 IndexType 枚舉
                    return obj.value
                return obj
            
            # 處理報告中的所有數據
            processed_report = convert_to_serializable(report)
            
            # 添加報告元數據
            processed_report['metadata'] = {
                'generation_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'index_type': index_type.value,
                'analysis_period': {
                    'start': period_start,
                    'end': period_end
                }
            }
                
            # 儲存為 JSON
            with open(save_path, 'w', encoding='utf-8') as f:
                json.dump(processed_report, f, ensure_ascii=False, indent=4)
                
            self.logger.info(f"已儲存產業分析報告: {save_path}")
            return save_path
                
        except Exception as e:
            self.logger.error(f"儲存產業分析報告時發生錯誤: {str(e)}")
            print(f"錯誤詳情: {str(e)}")  # 新增錯誤詳情輸出
            raise
    
    def _preprocess_report_for_json(self, report: Dict) -> Dict:
        """預處理報告數據以便JSON序列化"""
        def convert_value(value):
            if isinstance(value, (np.int32, np.int64)):
                return int(value)
            elif isinstance(value, (np.float32, np.float64)):
                return float(value)
            elif isinstance(value, np.ndarray):
                return value.tolist()
            elif isinstance(value, (datetime, pd.Timestamp)):
                return value.strftime('%Y-%m-%d')
            elif isinstance(value, dict):
                return {k: convert_value(v) for k, v in value.items()}
            elif isinstance(value, (list, tuple)):
                return [convert_value(i) for i in value]
            elif isinstance(value, (pd.Series, pd.DataFrame)):
                return convert_value(value.to_dict())
            elif pd.isna(value):
                return None
            return value
    
        processed = {k: convert_value(v) for k, v in report.items()}
        return processed

    def get_available_industries(self):
        """獲取所有可分析的產業列表"""
        industries = set()
        for index_name in self.industry_index['指數名稱'].unique():
            base_name = index_name.replace('類報酬指數', '').replace('類指數', '').strip()
            industries.add(base_name)
        return sorted(list(industries))
    
    def get_industry_selection(self, available_industries):
        """處理產業選擇邏輯"""
        while True:
            try:
                choice = input("\n請輸入產業編號或名稱: ").strip()
                if choice.isdigit() and 1 <= int(choice) <= len(available_industries):
                    return available_industries[int(choice)-1]
                elif choice in available_industries:
                    return choice
                else:
                    print("無效的選擇，請重新輸入！")
            except ValueError:
                print("無效的輸入，請重新選擇！")

    def check_existing_correlation_file(self, date: datetime, frequency: TimeFrequency) -> bool:
        """檢查相關性矩陣檔案是否已存在"""
        date_str = date.strftime('%Y%m%d')
        
        # 根據頻率決定檢查路徑
        if frequency == TimeFrequency.WEEKLY:
            check_dir = self.base_path / "industry_correlation/weekly"
        else:
            check_dir = self.base_path / "industry_correlation/monthly"
            
        filename = f"industry_correlation_{date_str}.csv"
        return (check_dir / filename).exists()

    def generate_correlation_series(self, frequency: TimeFrequency,
                                 start_date: str = None,
                                 end_date: str = None,
                                 progress_bar=None,
                                 force_update: bool = False):
        """生成並儲存時間序列相關性數據 (加入檢查機制)"""
        try:
            data = self.industry_index.copy()
            data['日期'] = pd.to_datetime(data['日期'])
            
            if start_date:
                data = data[data['日期'] >= pd.to_datetime(start_date)]
            if end_date:
                data = data[data['日期'] <= pd.to_datetime(end_date)]
                
            if frequency == TimeFrequency.WEEKLY:
                dates = data.groupby(pd.Grouper(key='日期', freq='W'))['日期'].last().dropna()
            else:
                dates = data.groupby(pd.Grouper(key='日期', freq='M'))['日期'].last().dropna()
            
            skipped_count = 0
            generated_count = 0
            
            for date in dates:
                if pd.notnull(date):
                    # 檢查檔案是否已存在
                    if not force_update and self.check_existing_correlation_file(date, frequency):
                        skipped_count += 1
                    else:
                        self.generate_correlation_matrix(date)
                        generated_count += 1
                        
                    if progress_bar is not None:
                        progress_bar.update(1)
            
            # 回報處理結果
            self.logger.info(f"處理完成: 新生成 {generated_count} 個檔案, 略過 {skipped_count} 個已存在的檔案")
            return generated_count, skipped_count
                        
        except Exception as e:
            self.logger.error(f"生成相關性數據時發生錯誤: {str(e)}")
            raise

In [4]:
def initialize_system():
    """初始化系統設定"""
    logging.getLogger().setLevel(logging.ERROR)
    base_path = Path("D:/Min/Python/Project/FA_Data")
    
    # 定義所需目錄
    directories = {
        'weekly': base_path / "industry_correlation/weekly",
        'monthly': base_path / "industry_correlation/monthly",
        'price_index': base_path / "industry_analysis/price_index",
        'return_index': base_path / "industry_analysis/return_index"
    }
    
    # 建立目錄
    for dir_path in directories.values():
        dir_path.mkdir(parents=True, exist_ok=True)
        
    return base_path, directories

def test_directory_permissions(directories):
    """測試目錄寫入權限"""
    try:
        test_file = directories['weekly'] / "test.txt"
        test_file.write_text("test")
        test_file.unlink()
        print("目錄寫入權限測試成功\n")
        return True
    except Exception as e:
        print(f"警告: 目錄寫入權限測試失敗: {str(e)}\n")
        return False

def process_correlation_data(analyzer, frequency: TimeFrequency, start_date: str, end_date: str, 
                           total_periods: int, desc: str):
    """處理相關性數據"""
    with tqdm(total=total_periods, desc=desc) as pbar:
        generated, skipped = analyzer.generate_correlation_series(
            frequency,
            start_date=start_date,
            end_date=end_date,
            progress_bar=pbar
        )
    return generated, skipped

def analyze_single_industry(analyzer, target_industry: str, start_date: str, end_date: str):
    """分析單一產業"""
    reports = {}
    for index_type in [IndexType.RETURN, IndexType.PRICE]:
        print(f"\n使用{index_type.value}進行分析...")
        report = analyzer.generate_enhanced_report(
            target_industry,
            start_date=start_date,
            end_date=end_date,
            index_type=index_type
        )
        
        if report:
            save_path = analyzer.save_industry_report(
                report,
                target_industry,
                index_type
            )
            reports[index_type] = {'report': report, 'path': save_path}
            print(f"\n{index_type.value}報告已儲存至: {save_path}")
    
    return reports

def display_comparison_analysis(reports, target_industry, start_date, end_date):
    """顯示比較分析結果"""
    print(f"\n=== {target_industry} 產業分析比較 ({start_date} 至 {end_date}) ===")
    
    for index_type, report_data in reports.items():
        if report_data and 'report' in report_data:
            report = report_data['report']
            print(f"\n【{index_type.value}分析結果】")
            
            # 顯示風險分析
            if 'risk_analysis' in report:
                risk = report['risk_analysis']
                ratios = risk.get('ratios', {})
                print(f"風險指標:")
                print(f"- 年化報酬率: {ratios.get('annual_return', 0):.2%}")
                print(f"- 年化波動率: {ratios.get('annual_volatility', 0):.2%}")
                print(f"- 夏普比率: {ratios.get('sharpe_ratio', 0):.2f}")
            
            # 顯示趨勢分析
            if 'time_series_analysis' in report:
                trend = report['time_series_analysis'].get('trend', {})
                print(f"\n趨勢分析:")
                print(f"- 趨勢方向: {trend.get('trend_direction', 'N/A')}")
                print(f"- 趨勢強度: {trend.get('trend_strength', 0):.2f}")
            
            # 顯示投資建議
            if 'investment_suggestions' in report:
                suggestions = report['investment_suggestions']
                print(f"\n投資建議:")
                print(f"- 風險評估: {suggestions.get('risk_assessment', '')}")
                print(f"- 進場建議: {suggestions.get('timing_suggestions', '')}")
                print(f"- 持倉建議: {suggestions.get('position_suggestions', '')}")

def get_analysis_mode():
    """獲取分析模式選擇"""
    while True:
        mode = input("\n請選擇分析模式 (1/2): ").strip()
        if mode in ['1', '2']:
            return mode
        print("無效的選擇，請輸入 1 或 2")

In [5]:
if __name__ == "__main__":
    try:
        # 1. 系統初始化
        base_path, directories = initialize_system()
        
        print("開始執行產業分析...\n")
        for name, path in directories.items():
            print(f"{name} 儲存路徑: {path}")
        print()
        
        # 2. 權限測試
        if not test_directory_permissions(directories):
            sys.exit(1)
            
        # 3. 初始化分析器與參數
        analyzer = IndustryAnalysisSystem()
        start_date = '2023-01-01'
        end_date = analyzer.industry_index['日期'].max().strftime('%Y-%m-%d')
        
        # 3.1 顯示產業指數配對驗證結果
        validation_results = analyzer.validate_index_pairs()
        print("\n=== 產業指數配對狀態 ===")
        if validation_results['missing_pairs']:
            print("\n缺少配對的產業:")
            for industry in validation_results['missing_pairs']:
                print(f"- {industry}")
        
        if validation_results['incomplete_pairs']:
            print("\n不完整配對的產業:")
            for pair in validation_results['incomplete_pairs']:
                print(f"- {pair['industry']} (價格指數: {'有' if pair['has_price'] else '無'}, "
                      f"報酬指數: {'有' if pair['has_return'] else '無'})")
        
        if validation_results['special_indices']:
            print("\n特殊指數產業:")
            for special in validation_results['special_indices']:
                print(f"- {special['industry']} (槓桿指數: {'有' if special['has_leverage'] else '無'}, "
                      f"反向指數: {'有' if special['has_inverse'] else '無'})")
        
        # 4. 選擇分析模式
        print("\n=== 分析模式選擇 ===")
        print("1: 單一產業分析")
        print("2: 所有產業分析")
        analysis_mode = get_analysis_mode()
        
        # 5. 處理相關性數據
        print("\n開始處理週度資料...")
        weekly_generated, weekly_skipped = process_correlation_data(
            analyzer, TimeFrequency.WEEKLY, start_date, end_date, 97, "週度資料處理"
        )
        print(f"\n週度資料處理完成:")
        print(f"- 新生成: {weekly_generated} 個檔案")
        print(f"- 略過已存在: {weekly_skipped} 個檔案")
        
        print("\n開始處理月度資料...")
        monthly_generated, monthly_skipped = process_correlation_data(
            analyzer, TimeFrequency.MONTHLY, start_date, end_date, 23, "月度資料處理"
        )
        print(f"\n月度資料處理完成:")
        print(f"- 新生成: {monthly_generated} 個檔案")
        print(f"- 略過已存在: {monthly_skipped} 個檔案")
        
        # 6. 產業分析
        print("\n開始產業分析...")
        
        if analysis_mode == "1":
            # 單一產業分析模式
            available_industries = analyzer.get_available_industries()
            print("\n可分析的產業列表：")
            for i, industry in enumerate(available_industries, 1):
                print(f"{i:2d}: {industry}")
                # 顯示該產業有哪些指數類型
                industry_indices = analyzer.industry_mapping.get(industry, {})
                index_types = []
                if industry_indices.get(IndexType.PRICE):
                    index_types.append("價格指數")
                if industry_indices.get(IndexType.RETURN):
                    index_types.append("報酬指數")
                if industry_indices.get(IndexType.LEVERAGE):
                    index_types.append("槓桿指數")
                if industry_indices.get(IndexType.INVERSE):
                    index_types.append("反向指數")
                if index_types:
                    print(f"     可用指數: {', '.join(index_types)}")
            
            target_industry = analyzer.get_industry_selection(available_industries)
            print(f"\n已選擇分析 {target_industry} 產業")
            
            reports = analyze_single_industry(analyzer, target_industry, start_date, end_date)
            display_comparison_analysis(reports, target_industry, start_date, end_date)
            
        else:
            # 所有產業分析模式
            print("\n開始分析所有產業...")
            industry_reports = analyzer.analyze_all_industries(
                start_date=start_date,
                end_date=end_date
            )

            index_type_names = {
                'price': '價格指數',
                'return': '報酬指數',
                'leverage': '槓桿指數',
                'inverse': '反向指數'
            }
            
            print("\n============ 產業分析報告總覽 ============")
            for industry_name, reports in industry_reports.items():
                print(f"\n產業：{industry_name}")
                for index_type, report in reports.items():
                    if report:
                        risk_analysis = report.get('risk_analysis', {})
                        time_series = report.get('time_series_analysis', {})
                        # 修改這行，直接顯示index_type
                        print(f"- {index_type}分析完成")  
                        print(f"  夏普比率: {risk_analysis.get('ratios', {}).get('sharpe_ratio', 0):.2f}")
                        print(f"  趨勢方向: {time_series.get('trend', {}).get('trend_direction', 'N/A')}")
                        
                # 顯示特殊指數資訊
                industry_indices = analyzer.industry_mapping.get(industry_name, {})
                if industry_indices.get(IndexType.LEVERAGE) or industry_indices.get(IndexType.INVERSE):
                    print("  特殊指數:", end=" ")
                    if industry_indices.get(IndexType.LEVERAGE):
                        print("槓桿指數", end=" ")
                    if industry_indices.get(IndexType.INVERSE):
                        print("反向指數")
                    print()
            
            print("\n====================================")
            
    except Exception as e:
        print(f"\n錯誤: {str(e)}")
        print(f"錯誤詳情:\n{traceback.format_exc()}")

開始執行產業分析...

weekly 儲存路徑: D:\Min\Python\Project\FA_Data\industry_correlation\weekly
monthly 儲存路徑: D:\Min\Python\Project\FA_Data\industry_correlation\monthly
price_index 儲存路徑: D:\Min\Python\Project\FA_Data\industry_analysis\price_index
return_index 儲存路徑: D:\Min\Python\Project\FA_Data\industry_analysis\return_index

目錄寫入權限測試成功


=== 產業指數配對狀態 ===

不完整配對的產業:
- 塑膠化工 (價格指數: 有, 報酬指數: 無)
- 機電 (價格指數: 有, 報酬指數: 無)
- 水泥窯製 (價格指數: 有, 報酬指數: 無)

特殊指數產業:
- 金融 (槓桿指數: 有, 反向指數: 有)
- 電子 (槓桿指數: 有, 反向指數: 有)

=== 分析模式選擇 ===
1: 單一產業分析
2: 所有產業分析



請選擇分析模式 (1/2):  2



開始處理週度資料...


週度資料處理: 107it [00:00, 116.57it/s]                                                                                



週度資料處理完成:
- 新生成: 7 個檔案
- 略過已存在: 100 個檔案

開始處理月度資料...


月度資料處理: 25it [00:01, 18.56it/s]                                                                                  



月度資料處理完成:
- 新生成: 11 個檔案
- 略過已存在: 14 個檔案

開始產業分析...

開始分析所有產業...

============ 產業分析報告總覽 ============

產業：貿易百貨
- price分析完成
  夏普比率: -0.19
  趨勢方向: upward
- return分析完成
  夏普比率: 0.11
  趨勢方向: upward

產業：汽車
- price分析完成
  夏普比率: 0.38
  趨勢方向: upward
- return分析完成
  夏普比率: 0.50
  趨勢方向: upward

產業：塑膠
- price分析完成
  夏普比率: -1.95
  趨勢方向: downward
- return分析完成
  夏普比率: -1.82
  趨勢方向: downward

產業：觀光
- price分析完成
  夏普比率: 2.32
  趨勢方向: upward
- return分析完成
  夏普比率: 2.39
  趨勢方向: upward

產業：水泥窯製
- price分析完成
  夏普比率: -0.19
  趨勢方向: downward
- return分析完成
  夏普比率: 0.00
  趨勢方向: N/A

產業：橡膠
- price分析完成
  夏普比率: 0.43
  趨勢方向: upward
- return分析完成
  夏普比率: 0.59
  趨勢方向: upward

產業：電子類兩倍槓桿指數
- price分析完成
  夏普比率: 0.00
  趨勢方向: N/A
- return分析完成
  夏普比率: 0.00
  趨勢方向: N/A

產業：綠能環保
- price分析完成
  夏普比率: 0.19
  趨勢方向: upward
- return分析完成
  夏普比率: 0.35
  趨勢方向: upward

產業：機電
- price分析完成
  夏普比率: 1.66
  趨勢方向: upward
- return分析完成
  夏普比率: 0.00
  趨勢方向: N/A

產業：觀光餐旅
- price分析完成
  夏普比率: -0.93
  趨勢方向: downward
- return分析完成
  夏普比率: -0.78
  趨勢方向: downward
